# Summary grading - Example for setting up feedback task with Log10

## Install dependencies

In [31]:
!pip install openai
!pip install -U log10-io
!pip install magentic

### Set up env vars

In [32]:
import os
os.environ["OPENAI_API_KEY"] = "" #replace with your key

os.environ["LOG10_URL"]="https://log10.io"
os.environ["LOG10_ORG_ID"]="" #replace with your credentials from Log10
os.environ["LOG10_TOKEN"]="" #replace with your token from Log10

In [33]:
import asyncio
import openai
from magentic import AsyncStreamedStr, chatprompt, SystemMessage, UserMessage
import uuid

from log10.load import log10, log10_session, last_completion_response
log10(openai)

In [34]:
# Tag selector below is used for:
# 1) Selecting which logs to show in the Feedback inbox in the GUI
# 2) Tagging new logs/completions to attach feedback to
task_tag_selector = "summary-grading-cnndm-SDK-test-6"

## Create task

In [35]:
from log10.feedback.feedback import Feedback
from log10.feedback.feedback_task import FeedbackTask

task = FeedbackTask().create(
    name="Summary grading task",
    task_schema={
        "$schema": "http://json-schema.org/draft-07/schema#",
        "title": "Summary Evaluation",
        "description": "Evaluates the quality of a summary based on four key aspects: Relevance, Coherence, Consistency, and Fluency.",
        "type": "object",
        "properties": {
            "Coherence": {
                "type": "integer",
                "minimum": 1,
                "maximum": 7,
                "description": """Coherence
For this axis, answer the question “how coherent is the summary on its own?” A summary is
coherent if, when read by itself, it’s easy to understand and free of English errors. A summary is
not coherent if it’s difficult to understand what the summary is trying to say. Generally, it’s more
important that the summary is understandable than it being free of grammar errors.
Rubric:
Score of 1: The summary is impossible to understand.
Score of 4: The summary has mistakes or confusing phrasing that make it a bit hard to understand.
Score of 7: The summary is perfectly clear.""",
            },
            "Accuracy": {
                "type": "integer",
                "minimum": 1,
                "maximum": 7,
                "description": """Accuracy
For this axis, answer the question “does the factual information in the summary accurately match
the post?” A summary is accurate if it doesn’t say things that aren’t in the article, it doesn’t mix up
people, and generally is not misleading. If the summary says anything at all that is not mentioned
in the post or contradicts something in the post, it should be given a maximum score of 5. (If you
are confused about how to use ‘6’, see the FAQ!)
Rubric:
Score of 1: The summary is completely wrong, made up, or exactly contradicts what is written in
the post.
Score of 4: The summary says at least one substantial thing that is not mentioned in the post, or
that contradicts something in the post.
(Score of 5: The summary says anything, no matter how small, that is not mentioned in the post,
or that contradicts something in the post.)
Score of 7: The summary has no incorrect statements or misleading implications.""",
            },
            "Coverage": {
                "type": "integer",
                "minimum": 1,
                "maximum": 7,
                "description": """Coverage
For this axis, answer the question “how well does the summary cover the important information
in the post?” A summary has good coverage if it mentions the main information from the post
that’s important to understand the situation described in the post. A summary has poor coverage if
someone reading only the summary would be missing several important pieces of information
about the situation in the post. A summary with good coverage should also match the purpose of
the original post (e.g. to ask for advice).
Rubric:
Score of 1: The summary contains no information relevant to the post.
Score of 4: The summary is missing at least 1 important piece of information required to understand the situation.
Score of 7: The summary covers all of the important information required to understand the
situation.
""",
            },
            "Overall": {
                "type": "integer",
                "minimum": 1,
                "maximum": 7,
                "description": """Overall quality
For this axis, answer the question “how good is the summary overall at representing the post?”
This can encompass all of the above axes of quality, as well as others you feel are important. If
it’s hard to find ways to make the summary better, give the summary a high score. If there are lots
of different ways the summary can be made better, give the summary a low score.
Rubric:
Score of 1: The summary is terrible.
Score of 4: The summary is an okay representation of the post, but could be significantly improved.
Score of 7: The summary is an excellent representation of the post.
""",
            },
        },
        "required": ["Coherence", "Accuracy", "Coverage", "Overall"],
    },
    completion_tags_selector=[task_tag_selector], # 1) Selecting which logs to show in the Feedback inbox in the GUI
    instruction="""You are an evaluator of summaries of articles on reddit. You are tasked with grading the summaries for accuracy, coherence, coverage and overall.

Coherence
For this axis, answer the question “how coherent is the summary on its own?” A summary is
coherent if, when read by itself, it’s easy to understand and free of English errors. A summary is
not coherent if it’s difficult to understand what the summary is trying to say. Generally, it’s more
important that the summary is understandable than it being free of grammar errors.
Rubric:
Score of 1: The summary is impossible to understand.
Score of 4: The summary has mistakes or confusing phrasing that make it a bit hard to understand.
Score of 7: The summary is perfectly clear.

Accuracy
For this axis, answer the question “does the factual information in the summary accurately match
the post?” A summary is accurate if it doesn’t say things that aren’t in the article, it doesn’t mix up
people, and generally is not misleading. If the summary says anything at all that is not mentioned
in the post or contradicts something in the post, it should be given a maximum score of 5. (If you
are confused about how to use ‘6’, see the FAQ!)
Rubric:
Score of 1: The summary is completely wrong, made up, or exactly contradicts what is written in
the post.
Score of 4: The summary says at least one substantial thing that is not mentioned in the post, or
that contradicts something in the post.
(Score of 5: The summary says anything, no matter how small, that is not mentioned in the post,
or that contradicts something in the post.)
Score of 7: The summary has no incorrect statements or misleading implications.

Coverage
For this axis, answer the question “how well does the summary cover the important information
in the post?” A summary has good coverage if it mentions the main information from the post
that’s important to understand the situation described in the post. A summary has poor coverage if
someone reading only the summary would be missing several important pieces of information
about the situation in the post. A summary with good coverage should also match the purpose of
the original post (e.g. to ask for advice).
Rubric:
Score of 1: The summary contains no information relevant to the post.
Score of 4: The summary is missing at least 1 important piece of information required to understand the situation.
Score of 7: The summary covers all of the important information required to understand the
situation.

Overall quality
For this axis, answer the question “how good is the summary overall at representing the post?”
This can encompass all of the above axes of quality, as well as others you feel are important. If
it’s hard to find ways to make the summary better, give the summary a high score. If there are lots
of different ways the summary can be made better, give the summary a low score.
Rubric:
Score of 1: The summary is terrible.
Score of 4: The summary is an okay representation of the post, but could be significantly improved.
Score of 7: The summary is an excellent representation of the post.""",
)

### Test prompt for generating the summary

In [46]:
@chatprompt(
    SystemMessage("Create a concise summary of the following article"),
    UserMessage(" {article}. \n\n\
Aim to make the summary perfectly clear (coherent), \
with no incorrect statements or misleading implications (accurate), \
covering all the important information required to understand the situation (high coverage) \
and an excellent representation of the post (high quality).")
)
async def create_message(article: str) -> AsyncStreamedStr:  # ruff: ignore
  ...

@chatprompt(
    SystemMessage("Create a concise summary of the following article"),
    UserMessage(" {article}. \n\n\
Aim to make the summary perfectly clear (coherent), \
with no incorrect statements or misleading implications (accurate), \
covering all the important information required to understand the situation (high coverage) \
and an excellent representation of the post (high quality).")
)
def create_sync_message(article: str) -> str:  # ruff: ignore
  ...


## Generate Example 1

In [38]:
uuid_str = str(uuid.uuid4())

async def main():
  # Example 1
  article = """Newcastle stand-in skipper Moussa Sissoko is facing disciplinary action after he was sent off following a reckless challenge on Liverpool midfielder Lucas Leiva during Monday's 2-0 defeat at Anfield.\n\nThe France international was given a second yellow card for the offence, but head coach John Carver feels it should have been a straight red.\n\n'The club will deal with that situation,' he said when asked if Sissoko - who is now banned for two matches - would be punished.\n\nLiverpool midfielder Lucas Leiva clutches his leg after Moussa Sissoko's tackle at Anfield\n\nSissoko hands the captain's armband to boss John Carver as he leaves the pitch after being sent off\n\n'He knows he was wrong. He was fortunate not to get a straight red and he agreed with me.\n\n'He apologised afterwards to Lucas, which was important.\n\n'But you think captains would lead by example. We have to improve our discipline. I will be looking at that.'\n\nMeanwhile, Carver says Newcastle cannot rely on the shortcomings of others to preserve their Premier League status.\n\nThe Magpies are the division's most out-of-form side having lost five on the spin, scoring just one goal along the way.\n\nLiverpool's players surround Lucas following Sissoko's dangerous tackle during Monday night's game\n\nRaheem Sterling bends the ball past Tim Krul to open the scoring in Liverpool's 2-0 win against Newcastle\n\nThey are nine points clear of danger with six matches to play, but Carver says it's about time they started helping themselves, starting with Sunday's visit of Spurs.\n\n'These two home games (Spurs followed by Swansea) are massive for us. I'm not bothered about performances, we need results,' he said.\n\n'I'm not worrying about that (relegation) at the moment, and the good thing is we have four games at home.\n\n'But we need to start winning now. We can't rely on others teams. We can't afford to ease off, I have always said that.\n\n'We have gone through a rough spell. It's down to me now to get players in right frame of mind.'\n\nNewcastle's players appear dejected as Joe Allen celebrates scoring Liverpool's second goal at Anfield"""

  with log10_session(tags=[task_tag_selector, uuid_str]): # remember to use unique uuid here to be able to assign the feedback in the next step
      output = await create_message(article)
      async for chunk in output:
          print(chunk, end="", flush=True)


await main()

Newcastle stand-in captain Moussa Sissoko faces potential disciplinary action from his club after receiving a second yellow card for a reckless challenge on Liverpool's Lucas Leiva in a recent 2-0 loss at Anfield. Sissoko, who subsequently apologized to Leiva, acknowledged the severity of his tackle, and Newcastle head coach John Carver suggested that the offense warranted a straight red card rather than just a booking. The incident contributes to ongoing disciplinary concerns for Newcastle, with Carver emphasizing the need for improved leadership and discipline within the team. Following the incident, Sissoko is set to serve a two-match ban.

Beyond the field issues, Carver stressed the urgency of addressing Newcastle's current form in the Premier League. The team has encountered a troubling streak, losing five consecutive matches and scoring only once during these games. Despite holding a nine-point buffer above the relegation zone with six games remaining, Carver highlighted the cri

### Assign Feedback on Example 1 via SDK/API

In [39]:
task_id = task.json()["id"]


Feedback().create(
    task_id=task_id,
    values={            #Dummy ratings
        "Overall": 3,
        "Accuracy": 5,
        "Coverage": 4,
        "Coherence": 2,
        },
    completion_tags_selector=[uuid_str, task_tag_selector], # 2) Tagging new logs/completions to attach feedback to
    comment="Misleading: \"Carver admits he is only concerned with results and not performances\" understood as if critics of monday's match but it's said for the following matches.\n\n13th??\n\nDoesnt properly address the teams, the match, the result, 2nd yellow card and therefore sent off, etc."
)

<Response [200 OK]>

## Generate Example 2

In [40]:
uuid_str = str(uuid.uuid4())

async def main():
  # Example 2
  article = """More than 70,000 people across the world have signed a petition demanding McDonald's remove spikes that deter the homeless from sleeping outside one of its restaurants.\n\nThe fast food giant has caused fury after installing metal studs outside its branch in Leeds city centre, which critics say are there to stop people  sleeping rough.\n\nMcDonald's has defended the use of the spikes, claiming they were installed two years ago in an attempt to stop anti-social behaviour, not target the homeless.\n\nUproar: A petition has been started to get the spikes removed from outside this Leeds McDonald's and 70,000 have backed the campaign\n\nBut critics say they have been put in because they are uncomfortable to sleep on and installed on areas which used to be benches.\n\nIsobella DeMartino set up the petition to have them removed and has received support from across the globe. \n\nRow: Critics say the spikes were installed to target the homeless but McDonald's say it is only about anti-social behaviour\n\n'Anti-homeless spikes show complete disregard and lack of respect for homeless people who struggle every day,' she said.\n\n'Spikes have been installed, presumably to prevent people sleeping there.\n\n'The fact that the spikes exist shows how little people care about homeless people.'\n\nShe added: 'Every time I'm in Leeds I see many ignore homeless people, not even bothering to smile at them let alone try to help them or give charity. I cannot imagine how painful and lonely it must be to be homeless.\n\n'They deserve our attention and care, this issue cannot be ignored.'\n\nA spokeswoman for McDonald's said: 'The studs in question are not an 'anti- homeless' measure, as has been suggested.\n\n'They have been in place for around two years and were introduced Following discussions with the police, in response to issues we were experiencing with antisocial behaviour.\n\n'We welcome all customers through our doors and it is inaccurate to suggest a measure taken in response to tackling antisocial behaviour, is an attempt to deter homeless people.'"""

  with log10_session(tags=[task_tag_selector, uuid_str]):
      output = await create_message(article)
      async for chunk in output:
          print(chunk, end="", flush=True)


await main()

Over 70,000 people worldwide have signed a petition calling on McDonald's to remove metal spikes outside a Leeds city center restaurant, intended to prevent rough sleeping. Critics argue the spikes discourage homeless individuals from resting there, a claim McDonald's refutes, saying the spikes aim to curb antisocial behavior, not to target the homeless. The petition, initiated by Isobella DeMartino, has gained international support. McDonald's insists the studs are part of a broader strategy, developed in consultation with the police, to manage antisocial issues at the location.

### Assign Feedback via the UI
1. Log in to your log10.io account
2. Navigate to the Feedback tab in the left side panel
3. You should see the above example in the inbox. Click on it to assign feedback via the UI

## Generate Example 3

In [50]:
uuid_str = str(uuid.uuid4())
last_completion_url = "init"

#async def main():
#Using sync version for now to be able to get last_completion_url and extract the completion_id

from log10.load import OpenAI
client = OpenAI()

# Example 3
article = """A humble wooden deckchair recovered from the doomed Titanic has sold for \u00a3100,350 at auction.\n\nThe Nantucket wooden chair, which once sat on a first-class promenade of the ill-fated ship, was salvaged by a search team from the Atlantic Ocean after the Titanic sank in 1912. \n\nDubbed 'one of the rarest types of Titanic collectable', the chair is too fragile to sit on, but has been carefully preserved, having been owned by a British collector for the past 15 years.\n\nA 103-year-old deckchair recovered from the wreck of the Titanic is expected sold for \u00a3100,000 at auction\n\nThe ship's log records six or seven deckchairs being picked up and taken back to port in Halifax, Nova Scotia.\n\nOne was given by a crew member to Captain Julien Lemarteleur, who had previously worked on board the Mackay-Bennett. \n\nThe anonymous seller kept it by a large window overlooking the sea at his home on the south coast, never sat on it due to its fragile state and instead used it as a display item. \n\nIt was sold yesterday at Henry Aldridge and Son auction house in Devizes, Wiltshire, to a UK buyer. \n\nAndrew Aldridge, of auctioneers Henry Aldridge and Son, described the piece as one of 'the most important [bits] of Titanic memorabilia around'.  \n\nHe said: 'It is incredibly rare to find something of that size that was on Titanic, was salvaged and that still exists today.\n\nSix or seven recovered deckchairs were stacked on the deck of the Mackay-Bennett as it arrived in Nova Scotia\n\nTitanic (above) sank with the loss of 1,522 lives four hours after it struck on iceberg at 10.20pm on April 14\n\nIt has a five pointed star on top of it, which was the emblem of White Star Line - the company that owned the Titanic\n\nHe said that he was 'very, very pleased' with the price.\n\nThe chair is made of teak wood and has an adjustable footrest at the front. It is 57 inches long, 23 inches wide and 35 inches tall.\n\nIt has a five pointed star on top of it, which was the emblem of White Star Line - the company that owned the Titanic. \n\nUpon his death in 1973, Capt Lemarteleur gave the chair to colleague Captain Robin Lee. The current owner acquired it in 2001.\n\nSold with the chair was an in depth 'provenance package' that authenticates history and association with Titanic.\n\nCaptain Julien Lemarteleur (above), who collected the chair from the Mackay-Bennett more than 100 years ago\n\nIt includes a signed affidavit from Margarete Pennington, who was Mr Lemarteleur's housekeeper in the 1960s.\n\nIn it she states: 'I personally saw the deckchair that Captain Lemarteleur told me about on many occasions. He said that various pieces of debris taken from the ocean including this Titanic deckchair he acquired.\n\n'I recognise the chair as the same chair I saw in the captain's garage and of which the captain identified as coming from the Titanic.'\n\nCraig Sopin, 57, a lawyer from Philadelphia, owns one of the world's largest collections of Titanic memorabilia.\n\nHe also authenticated the chair and said: 'It isn't often that one has the opportunity to acquire such an important part of Titanic's story.\n\n'For such a significant item to have such a meticulous chain of custody is nothing short of astonishing. Its history can be traced from the auction house back to Titanic itself.'\n\nTitanic sank with the loss of 1,522 lives four hours after it struck on iceberg at 10.20pm on April 14.\n\nA ship, the Carpathia, picked up more than 700 survivors in lifeboats hours later and took them back to New York.\n\nWhite Star Line chartered four ships, including the Halifax-based cable-laying vessel the Mackay-Bennett, to recover the bodies."""

with log10_session(tags=[task_tag_selector, uuid_str]) as session:
    response = create_sync_message(article)
    print(response)
    last_completion_url = session.last_completion_url()
    print(last_completion_url)

completion_id = last_completion_url.split("/")[-1]
#await main()

A deckchair from the Titanic, one of the few recovered after the ship sank in 1912, sold for £100,350 at auction. It was part of the Titanic memorabilia, regarded for its rarity and historical significance. Salvaged by a search team and previously owned by a British collector for 15 years, the teak wood chair is too fragile for use and has been maintained as a display item. Originating from the Titanic, which sank after striking an iceberg, resulting in 1,522 deaths, the chair has a significant provenance package confirming its authenticity. It was sold to a UK buyer by Henry Aldridge and Son auction house in Devizes, Wiltshire.
https://log10.io/app/log10/completions/e0ed3077-f792-4b16-a3b6-b782e5c5fe48


In [51]:
print(completion_id)

e0ed3077-f792-4b16-a3b6-b782e5c5fe48


In [42]:
# # async way

# uuid_str = str(uuid.uuid4())
# last_completion_url = "init"

# async def main():
#   # Example 3
#   article = "That cat ran over the mouse" # for quick testing

#   with log10_session(tags=[task_tag_selector, uuid_str]):
#       output = await create_message(article)
#       async for chunk in output:
#           print(chunk, end="", flush=True)
#       global last_completion_url
#       last_completion_url = session.last_completion_url()

# await main()

#article = """A humble wooden deckchair recovered from the doomed Titanic has sold for \u00a3100,350 at auction.\n\nThe Nantucket wooden chair, which once sat on a first-class promenade of the ill-fated ship, was salvaged by a search team from the Atlantic Ocean after the Titanic sank in 1912. \n\nDubbed 'one of the rarest types of Titanic collectable', the chair is too fragile to sit on, but has been carefully preserved, having been owned by a British collector for the past 15 years.\n\nA 103-year-old deckchair recovered from the wreck of the Titanic is expected sold for \u00a3100,000 at auction\n\nThe ship's log records six or seven deckchairs being picked up and taken back to port in Halifax, Nova Scotia.\n\nOne was given by a crew member to Captain Julien Lemarteleur, who had previously worked on board the Mackay-Bennett. \n\nThe anonymous seller kept it by a large window overlooking the sea at his home on the south coast, never sat on it due to its fragile state and instead used it as a display item. \n\nIt was sold yesterday at Henry Aldridge and Son auction house in Devizes, Wiltshire, to a UK buyer. \n\nAndrew Aldridge, of auctioneers Henry Aldridge and Son, described the piece as one of 'the most important [bits] of Titanic memorabilia around'.  \n\nHe said: 'It is incredibly rare to find something of that size that was on Titanic, was salvaged and that still exists today.\n\nSix or seven recovered deckchairs were stacked on the deck of the Mackay-Bennett as it arrived in Nova Scotia\n\nTitanic (above) sank with the loss of 1,522 lives four hours after it struck on iceberg at 10.20pm on April 14\n\nIt has a five pointed star on top of it, which was the emblem of White Star Line - the company that owned the Titanic\n\nHe said that he was 'very, very pleased' with the price.\n\nThe chair is made of teak wood and has an adjustable footrest at the front. It is 57 inches long, 23 inches wide and 35 inches tall.\n\nIt has a five pointed star on top of it, which was the emblem of White Star Line - the company that owned the Titanic. \n\nUpon his death in 1973, Capt Lemarteleur gave the chair to colleague Captain Robin Lee. The current owner acquired it in 2001.\n\nSold with the chair was an in depth 'provenance package' that authenticates history and association with Titanic.\n\nCaptain Julien Lemarteleur (above), who collected the chair from the Mackay-Bennett more than 100 years ago\n\nIt includes a signed affidavit from Margarete Pennington, who was Mr Lemarteleur's housekeeper in the 1960s.\n\nIn it she states: 'I personally saw the deckchair that Captain Lemarteleur told me about on many occasions. He said that various pieces of debris taken from the ocean including this Titanic deckchair he acquired.\n\n'I recognise the chair as the same chair I saw in the captain's garage and of which the captain identified as coming from the Titanic.'\n\nCraig Sopin, 57, a lawyer from Philadelphia, owns one of the world's largest collections of Titanic memorabilia.\n\nHe also authenticated the chair and said: 'It isn't often that one has the opportunity to acquire such an important part of Titanic's story.\n\n'For such a significant item to have such a meticulous chain of custody is nothing short of astonishing. Its history can be traced from the auction house back to Titanic itself.'\n\nTitanic sank with the loss of 1,522 lives four hours after it struck on iceberg at 10.20pm on April 14.\n\nA ship, the Carpathia, picked up more than 700 survivors in lifeboats hours later and took them back to New York.\n\nWhite Star Line chartered four ships, including the Halifax-based cable-laying vessel the Mackay-Bennett, to recover the bodies."""

### Use AutoFeedback to assign Feedback

In [52]:
from log10.feedback._summary_feedback_utils import flatten_messages, summary_feedback_llm_call
from log10.feedback.autofeedback import AutoFeedbackICL

num_few_shot_examples = 1 # recommend setting to 5

# Setup AutoFeedbackICL
auto_feedback_icl = AutoFeedbackICL(task_id, num_samples=num_few_shot_examples, predict_func=summary_feedback_llm_call)

# Get completion_id to predict feedback on
#completion_id = "e77c023f-b847-4d16-9509-1e62e22c84dc"
autofeedback_prediction = auto_feedback_icl.predict(completion_id=completion_id)

print(autofeedback_prediction)


{"note": "The summary effectively condenses the main points of the article, maintaining a focus on the sale and historical significance of the Titanic deckchair. It is coherent, accurate, and covers the essential details without overwhelming the reader with minutiae.", "axes": {"overall": "7", "accuracy": "7", "coverage": "6", "coherence": "7"}}


In [60]:
import json

# covert AutoFeedback to json
af_pred = json.loads(autofeedback_prediction)
print(af_pred["axes"], af_pred["note"])


{'overall': '7', 'accuracy': '7', 'coverage': '6', 'coherence': '7'} The summary effectively condenses the main points of the article, maintaining a focus on the sale and historical significance of the Titanic deckchair. It is coherent, accurate, and covers the essential details without overwhelming the reader with minutiae.


In [58]:
print(uuid_str, task_tag_selector)

87b8ff31-1dc2-4763-bdf5-e7ec40fd4196 summary-grading-cnndm-SDK-test-6


In [61]:
# Use the above AutoFeedback to populate the feedback
task_id = task.json()["id"]

Feedback().create(
    task_id=task_id,
    values={
        "Overall": int(af_pred["axes"]["overall"]),
        "Accuracy": int(af_pred["axes"]["accuracy"]),
        "Coverage": int(af_pred["axes"]["coverage"]),
        "Coherence": int(af_pred["axes"]["coherence"]),
        },
    completion_tags_selector=[uuid_str, task_tag_selector], # 2) Tagging new logs/completions to attach feedback to
    comment=af_pred["note"]
)

#This will appear in the Feedback stream on the Feedback tab

<Response [200 OK]>

## Talk to us / Coming soon
1. Use Log10's managed AutoFeedback service to automatically assign feedback to new generations from the LLM
2. Get summary stats over feedback (human + AutoFeedback)
3. Set up alerts over AutoFeedback
4. Curate datasets for fine-tuning or prompt engineering to improve accuracy and customize per customer